# Tarea 4 - MAN3160



**Profesor**: Denis Parra

**Ayudante**: Álvaro Labarca.


Esta tarea esta dividida en dos secciones. En la primera se evaluarán métodos de recomendación con métricas que no están enfocadas hacia la exactitud de las recomendaciones y la segunda sección es de utilizar Gru4Rec para generar un modelo de recomendación secuencial.

# Sección 1 - Métricas Beyond-Accuracy.

Hasta ahora, hemos visto una gran cantidad de formas de evaluar sistemas recomendadores, métricas de error, métricas de ranking, métricas de presición, etc.

Estas métricas se enfocan principalmente en evaluar si las recomendaciones entregadas se ajustan a las preferencias reales de los usuarios, ya sea prediciendo calificaciones cercanas a las dadas por el usuario o bien, recomendando elementos con los que el usuario realmente interactuó.

Sin embargo, ciertos estudios, como los llevados a cabo por Ziegler et al. (2005), Vargas y Castell (2011) y Lathia et al. (2010) proponen evaluar modelos con métricas denominadas en la literatura como métricas Beyond-Accuracy. Este tipo de métricas se enfocan en evaluar características de las recomendaciones que pueden llevar a una mayor satisfacción para el usuario o que se ajusten mejor a los objetivos de las plataformas.

En clases se mencionó dos de estas métricas: **Diversidad:** La cual mide qué tan "distintas" entre sí son las recomendaciones entregadas por el sistema y **novedad**, la cual busca que el sistema recomiende ítems que sean "novedosos", es decir, que recomiende ítems que sean poco probables que el usuario ya conozca.

Estas métricas han sido definidas de distintas maneras por distintos autores. Para el desarrollo de esta tarea, vamos a utilizar la **Diversidad de Ziegler** y la **Novedad de Vargas**.

Para esta sección, vamos a trabajar con el dataset de MovieLens.

## Descarga del dataset

In [1]:
!pip install wget
!pip install zipfile36
!pip3 install implicit --upgrade
!python -m wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!pip install deepctr-torch
!pip install tensorflow
!pip install lightfm


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
/Users/ldavico/.pyenv/versions/3.11.2/bin/python: No module named wget

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
!pip install wget
!pip install zipfile36
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
import zipfile
with zipfile.ZipFile("ml-100k.zip", 'r') as zip_ref:
    zip_ref.extractall(".")


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
--2024-05-14 21:34:45--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolviendo files.grouplens.org (files.grouplens.org)... 128.101.65.152
Conectando con files.grouplens.org (files.grouplens.org)[128.101.65.152]:80... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 4924029 (4.7M) [application/zip]
Grabando a: «ml-100k.zip.2»

ml-100k.zip.2       100%[===================>]   4.70M   804KB/s    en 6.4s    

2024-05-14 21:34:52 (751 KB/s) - «ml-100k.zip.2» guardado [4924029/4924029]



In [3]:
# Pueden importar las librerías que estimen necesarias.

import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import implicit
import numpy as np
import scipy.sparse as sparse
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from deepctr_torch.inputs import SparseFeat, VarLenSparseFeat
from sklearn.model_selection import train_test_split
import tensorflow as tf
import torch
import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation
from lightfm.evaluation import auc_score, precision_at_k

/opt/homebrew/lib/python3.11/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [4]:
dir_train = 'ml-100k'

# Generamos los títulos de las columnas del archivo items.

columns = ['itemid', 'title', 'release_date', 'video_release_date', \
           'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', \
           'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', \
           'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', \
           'Thriller', 'War', 'Western']

In [5]:
df = pd.read_csv(f'{dir_train}/u.data',
                         sep='\t',
                         names=['userid', 'itemid', 'rating', 'timestamp'],
                         header=None)

In [6]:
df_items = pd.read_csv(f'{dir_train}/u.item',
                        sep='|',
                        index_col=0,
                        names = columns,
                        header=None,
                        encoding='latin-1')

### Generación de vector latente basado en contenido.

Para la métrica de Diversidad de Ziegler, necesitamos un método para calcular la distancia entre dos ítems. Para esto, usaremos la información de contenido de ítems proporcionada por MovieLens para generar vectores latentes para cada ítem. En específico, utilizaremos la información del género de las películas y el año en que fueron estrenadas.

In [7]:
df_items.head()

,title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
itemid,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Primero, se obtiene el año de estreno a partir de la columna 'release_date'

In [8]:
df_items['release_date'] = pd.to_datetime(df_items['release_date'], format='%d-%b-%Y')

# Extract year and create a new column 'year'
df_items['year'] = df_items['release_date'].dt.year

Como la escala de la información del año es mucho más elevada que la información de género, usamos una normalización de Z-score para generar la columna 'normalized_year'

In [9]:
mean = df_items['year'].mean()
std = df_items['year'].std()

df_items['normalized_year'] = (df_items['year'] - mean) / std

In [10]:
df_items

,title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year,normalized_year
itemid,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,1995.0,0.393860
2,GoldenEye (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,1,0,0,1995.0,0.393860
3,Four Rooms (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1995.0,0.393860
4,Get Shorty (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1995.0,0.393860
5,Copycat (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1995.0,0.393860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,Mat' i syn (1997),1998-02-06,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1998.0,0.604334
1679,B. Monkey (1998),1998-02-06,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,1998.0,0.604334
1680,Sliding Doors (1998),1998-01-01,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1998.0,0.604334


In [11]:
# Eliminamos las filas con ítems que no tienen información de año

df_items.dropna(subset=['normalized_year'], inplace=True)
df_items = df_items.reset_index()
df = df[df['itemid'].isin(df_items['itemid'])]

In [12]:
contenido = df_items[['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', \
                      'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', \
                      'normalized_year']].values

In [13]:
# Utilizamos PCA20 para generar una representación latente de dimensión 5

pca20 = PCA(n_components=5).fit_transform(contenido)

In [14]:
# Generamos un diccionario {itemid: vector_latente}

item_feats = dict(zip(df_items['itemid'], pca20.tolist()))

In [15]:
columns_user = ['userid', 'age', 'gender', 'occupation', 'zip_code']

In [16]:
df_users = pd.read_csv(f'{dir_train}/u.user',
                        sep='|',
                        index_col=0,
                        names = columns_user,
                        header=None,
                        encoding='latin-1')

df_users = df_users.reset_index()
df_items = df_items.reset_index()

## Metricas

### Diversidad de Ziegler

El objetivo de la Diversidad de Ziegler es calcular la similaridad entre todos los elementos de una lista de recomendación. Mientras menor sea la similaridad total dentro de una lista, más diversa es.

In [17]:
# Generamos un diccionario {(item1, item2): Distancia} Para evitar calcular la misma distancia más de una vez

dist_dict = dict()

def similarity(rec):
    s = 0
    for m in rec:
        s += similarity_i(m, rec)
    return s/2

def similarity_i(m, rec):
    s = 0
    if len(rec) <= 1:
        return 0
    for m2 in rec:
        if m != m2:
            s += (1 - distance(m, m2))
    return s

def distance(i1, i2):
    if (i1, i2) in dist_dict.keys():
        return dist_dict[(i1, i2)]
    dist = pairwise_distances(np.array(item_feats[i1]).reshape(1,-1), np.array(item_feats[i2]).reshape(1,-1), metric='cosine')
    dist_dict[(i1, i2)] = dist
    dist_dict[(i2, i1)] = dist
    return dist

### Novedad de Vargas

Vargas y Castell (2011) definen un concepto de novedad basado en la probabilidad de que el usuario no haya visto ese ítem anteriormente. Esta probabilidad la plantean como el logaritmo de la popularidad inversa del ítem.

In [18]:
def popularity(iid):
    l = len(df[df['itemid'] == iid])
    return l/len(df)

dict_novelty = defaultdict(lambda: -100)
def novelty(rec):
    tot = 0
    for iid in rec:
        if dict_novelty[iid] == -100:
            novl = novelty_i(iid)
            dict_novelty[iid] = novl
            tot += novl
        else:
            tot += dict_novelty[iid]
    return tot/len(rec)

def novelty_i(iid):
    return -1*np.log2(popularity(iid))

### Metricas generales

In [19]:
# Definicion de métricas (No editar)
# Obtenido de https://gist.github.com/bwhite/3726239

def _precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    r = np.asarray(r) != 0
    out = [_precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)

    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

# Define function to obtain nDCG scores
def get_ndcg(surprise_predictions, k_highest_scores=None):
    """
    Calculates the ndcg (normalized discounted cumulative gain) from surprise predictions, using sklearn.metrics.ndcg_score and scipy.sparse

    Parameters:
    surprise_predictions (List of surprise.prediction_algorithms.predictions.Prediction): list of predictions
    k_highest_scores (positive integer): Only consider the highest k scores in the ranking. If None, use all.

    Returns:
    float in [0., 1.]: The averaged NDCG scores over all recommendations

    """

    from sklearn.metrics import ndcg_score
    from scipy import sparse

    uids = [int(p.uid) for p in surprise_predictions ]
    iids = [int(p.iid) for p in surprise_predictions ]
    r_uis = [p.r_ui for p in surprise_predictions ]
    ests = [p.est for p in surprise_predictions ]

    assert(len(uids) == len(iids) == len(r_uis) == len(ests) )

    sparse_preds = sparse.coo_matrix( (ests, (uids , iids )) )
    sparse_vals = sparse.coo_matrix( (r_uis, (uids , iids )) )

    dense_preds = sparse_preds.toarray()
    #print(dense_preds)
    dense_vals = sparse_vals.toarray()

    return ndcg_score(y_true= dense_vals , y_score= dense_preds, k=k_highest_scores, ignore_ties=True)

def get_AUC_at_k(model, k):

    auc = implicit.evaluation.AUC_at_k(model,
                                       user_item_matrix,
                                       user_item_matrix_test,
                                       show_progress=False,
                                       K=k
                                    )
    return auc

# Actividades

### Actividad 1: Cálculo de métricas Beyond-Accuracy.

* Elija y entrene 3 métodos vistos en clases (pueden ser cualquiera de los métodos de recomendación personalizada, ya sea de feedback implícito, explícito, híbrido, etc.)
* Elija 2 métricas de evaluación tradicional (pueden ser métricas de error, ranking, precision, etc.)
* Evalúe los métodos entrenados y genere una tabla que contenga los valores de las dos métricas elegidas por usted y las métricas de diversidad y novedad proporcionadas.
* Para las métricas Beyond-Accuracy, evalúe los sistemas con Novelty@k y Diversity@k con k = 10, 15, 20. Donde k es el largo de las listas de recomendación que se le entregan a los usuarios.

Tenga en cuenta los siguientes puntos.

* Dependiendo el método que elija, puede que tenga que realizar un pre-procesamiento distinto para los datos. Puede cambiar el dataset df utilizado por una de los splits predeterminados de MovieLens (ux.base). Si utiliza el dataset completo, recuerde generar el split train-test antes de entrenar. Si cambia el archivo, recuerde eliminar el item con el dato de fecha de estreno faltante (como se hizo al comienzo de este cuadernillo).
* Las funciones de métricas Beyond-Accuracy entregadas evalúan una única lista de recomendación. Para evaluar un sistema de recomendación, genere listas de recomendación para todos los usuarios del set de testeo y obtenga el promedio de todos los usuarios.
* Recuerde que la función entregada para la diversidad es una métrica de similaridad. Para sus análisis, debe tener en cuenta que un mayor valor implica una menor similaridad.
* Las funciones se definieron para recibir los ids de los items según están en el dataframe original (itemid). Tenga cuidado al manejar estos ids para evitar referirse a items equivocados.

Concluya a partir de los resultados de la tabla. Discuta por qué podría ser conveniente tener un nivel de Novelty y Diversity alto.

#### Respuesta:

Ingrese su respuesta continuación. Recuerde terminar su respuesta con una celda de texto con sus conclusiones y respuestas.

In [20]:
# Preparacion Datos ALS/BPR
train_dir = "ml-100k/u3.base"
test_dir = "ml-100k/u3.test"
train_file = pd.read_csv(train_dir, sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]
info_file = pd.read_csv('ml-100k/u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')
# Cambio los nombres de los archivos para un formato mas familiar

df_train = train_file.copy()

df_items = info_file.copy()

# Si el rating es >= 3 entonces es relevante, si no, 0

df_train.rating = [1 if x > 3 else 0 for x in df_train.rating]

# Creo los objetos que contendran la informacion de los usuarios e items
user_items = {}
itemset = set ()

# Itero sobre toda el df de entrenamiento y junto en un diccionario el id de un usuario
# Con sus items relevantes
for row in df_train.itertuples():
    if row[3]:
        if row[1] not in user_items:
            user_items[row[1]] = []

        user_items[row[1]].append(row[2])
        itemset.add(row[2])

# Cambio el formato del set de items para poder ordenarlo
itemset = np.sort(list(itemset))

# Creo una matriz de ceros con las dimensiones UxI
sparse_matrix = np.zeros((len(user_items), len(itemset)))

# Cambio los valores de los ceros por el de 1 cuando el item es relevante
for i, items in enumerate(user_items.values()):
    sparse_matrix[i] = np.isin(itemset, items, assume_unique = True).astype(int)

# Crear una matriz comprimida que utilice menos carga computacional
user_item_matrix = sparse.csr_matrix(sparse_matrix)

# Creo un diccionario con los ids de usuarios e items

user_ids = {key: i for i, key in enumerate(user_items.keys())}
items_ids = {key: i for i, key in enumerate(itemset)}

# Cargamos el dataser de prueba
df_test = pd.read_csv(f'{test_dir}',
                      sep='\t',
                      names=['userid', 'itemid', 'rating', 'timestamp'],
                      header=None)


# rating >= 3 es relevante (1) y rating menor a 3 es no relevante (0)
df_test.rating = [1 if x >=3 else 0 for x in df_test.rating ]


user_items_test = {}

for row in df_test.itertuples():
    if row[3] and row[1] in user_items:
        if row[1] not in user_items_test:
            user_items_test[row[1]] = []

        user_items_test[row[1]].append(row[2])

In [21]:
# Modelo ALS
# ALS
f = 15
rg = 0.01
model_als = implicit.als.AlternatingLeastSquares(factors=f, regularization=rg)
model_als.fit(user_item_matrix)
#als_mmap, als_ndcg, als_precision = evaluate_model2(model_als, 10)

# BPR
f = 70
it = 50
model_bpr = implicit.bpr.BayesianPersonalizedRanking(factors=f, iterations=it)
model_bpr.fit(user_item_matrix)
#bpr_mmap, bpr_ndcg, bpr_precision = evaluate_model2(model_bpr, 10)

/opt/homebrew/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 10 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [22]:
# Modelo light FM
dataset = Dataset()
# Se ajusta el objeto dataset con la información de usuarios e ítems

dataset.fit(users=df['userid'],
            items=df['itemid'])

# A partir de esto, podemos determinar fácilmente el número de usuarios e ítems únicos.
num_users, num_items = dataset.interactions_shape()
print(f'Num users: {num_users}, num_items: {num_items}.')
(interactions, weights) = dataset.build_interactions(df.iloc[:, 0:3].values)
train_interactions, test_interactions = cross_validation.random_train_test_split(
    interactions, test_percentage=0.25)

model1 = LightFM(no_components=15, loss='logistic')
model1.fit(interactions=train_interactions, epochs=5)

print(f'AUC score: {auc_score(model1, test_interactions, train_interactions=train_interactions).mean()}')
print(f'Precision at 20: {precision_at_k(model1, test_interactions, train_interactions=train_interactions, k=20).mean()}\n')

Num users: 943, num_items: 1681.
AUC score: 0.8629933595657349
Precision at 20: 0.1768822818994522



In [ ]:
#Modelo BPR

### Actividad 2: Accuracy vs. Beyond-Accuracy.

* Elija uno de los métodos que utilizó en la actividad 1 (no necesariamente tiene que ser el que entregó mejores resultados) y entrénelo modificando los valores de sus hiperparámetros.
* Elija una de las métricas tradicionales que utilizó en la actividad 1 y evalúe los modelos entrenados con distintos hiperparámetros en esta métrica en Accuracy@20 y Novelty@20.
* Genere gráficos que comparen el valor de la métrica tradicional (eje x) con las métrica beyond-accuracy (eje y). Por ejemplo, un gráfico de recall-Accuracy y otro de recall-Novelty.
* Cada gráfico debe tener al menos 5 puntos evaluados.

Concluya a partir de estos gráficos, discutiendo la forma de estos y lo que esto implica al generar un modelo recomendador.

#### Respuesta:

Ingrese su respuesta continuación. Recuerde terminar su respuesta con una celda de texto con sus conclusiones y respuestas.

### Actividad 3: Beyond-Accuracy vs. Beyond-Accuracy

Repita el procedimiento de la actividad 2, pero esta vez genere un gráfico de Novelty@20 Vs. Diversity@20.

A partir de este gráfico, concluya la correlación entre estas dos métricas y explique en sus palabras por qué cree que se da este efecto.

#### Respuesta:

Ingrese su respuesta continuación. Recuerde terminar su respuesta con una celda de texto con sus conclusiones y respuestas.

# Sección 2 - Recomendación secuencial.

En esta sección utilizaremos Gru4Rec, para entrenar un modelo de recomendación secuencial basado en sesiones.

## Entrenamiento de Gru4Rec

Seguiremos el mismo procedimiento utilizado en el tutorial de Gru4Rec para entrenar el modelo. Este proceso puede llegar a tomar más de 40 minutos, pero solo se debe realizar una vez, ya que los archivos generados se guardan en la carpeta en la que se está corriendo el cuadernillo, por lo que se recomienda ejecutar el código con anticipación.

In [88]:
!pip install py7zr
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113

In [ ]:
!git clone https://github.com/hidasib/GRU4Rec_PyTorch_Official --single-branch src

In [ ]:
# Descarga del dataset. Esto puede tomar hasta ~40 minutos, pero solo se debe realizar una vez

!curl https://s3-eu-west-1.amazonaws.com/yc-rdata/yoochoose-data.7z --output data.7z

In [ ]:
from py7zr import unpack_7zarchive
import shutil

shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
shutil.unpack_archive('data.7z', './rsc15')

**Impotante:** Es posible que al ejecutar el código, se encuentren con el error '*IndexError: tensors used as indices must be long, byte or bool tensors*'. Para arreglar este error, deben modificar la línea 23 del archivo evaluation.py por:

tscores = torch.diag(oscores[out_idxs.long()])

En el tutorial Gru4Rec podrán encontrar una explicación paso a paso.

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt

In [ ]:
!mkdir data

In [ ]:
src_path = './rsc15/'
dst_path   = './data/'

In [ ]:
def process(src_path, dst_path):
    data = pd.read_csv(src_path + 'yoochoose-clicks.dat', sep=',', header=None, usecols=[0,1,2], dtype={0:np.int32, 1:str, 2:np.int64})
    data.columns = ['SessionId', 'TimeStr', 'ItemId']

    # Convertimos el Timestamp en un objeto datetime
    data['Time'] = data.TimeStr.apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').timestamp())
    del(data['TimeStr'])

    # Tomamos las sesiones de largo > 1
    session_lengths = data.groupby('SessionId').size()  
    data = data[np.in1d(data.SessionId, session_lengths[session_lengths>1].index)]

    # Filtramos los items que aparecen menos de 5 veces.
    item_supports = data.groupby('ItemId').size()
    data = data[np.in1d(data.ItemId, item_supports[item_supports>=5].index)]

    # Volvemos a filtrar las sesiones de largo 1
    session_lengths = data.groupby('SessionId').size()
    data = data[np.in1d(data.SessionId, session_lengths[session_lengths>=2].index)]

    # Vamos a definir el set de testeo como todas las interacciones ocurridas el último día.
    tmax = data.Time.max()
    session_max_times = data.groupby('SessionId').Time.max()

    session_train = session_max_times[session_max_times < tmax-86400].index
    train = data[np.in1d(data.SessionId, session_train)]

    session_test = session_max_times[session_max_times >= tmax-86400].index
    test = data[np.in1d(data.SessionId, session_test)]
    test = test[np.in1d(test.ItemId, train.ItemId)]
    tslength = test.groupby('SessionId').size()
    test = test[np.in1d(test.SessionId, tslength[tslength>=2].index)]
    print('Test set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(test), test.SessionId.nunique(), test.ItemId.nunique()))
    test.to_csv(dst_path + 'test.tsv', sep='\t', index=False)

    # Validation: Vamos a usar el penúltimo día para el set de validación
    tmax = train.Time.max()
    session_max_times = train.groupby('SessionId').Time.max()
    session_train = session_max_times[session_max_times < tmax-86400].index
    session_valid = session_max_times[session_max_times >= tmax-86400].index

    train_tr = train[np.in1d(train.SessionId, session_train)]
    valid = train[np.in1d(train.SessionId, session_valid)]
    valid = valid[np.in1d(valid.ItemId, train_tr.ItemId)]
    tslength = valid.groupby('SessionId').size()
    valid = valid[np.in1d(valid.SessionId, tslength[tslength>=2].index)]
    print('Validation set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(valid), valid.SessionId.nunique(), valid.ItemId.nunique()))
    valid.to_csv(dst_path + 'validation.tsv', sep='\t', index=False)


    print('Train set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(train_tr), train_tr.SessionId.nunique(), train_tr.ItemId.nunique()))
    train_tr.to_csv(dst_path + 'train.tsv', sep='\t', index=False)

In [ ]:
process(src_path, dst_path)  # Toma ~ 10 minutos

In [ ]:
def train_fraction(df, fraction, prefix="train", path="./data/"):
    
    length = len(df['ItemId'])
    first_session = df.iloc[length - length//fraction].SessionId # Se toma el Id de la sesión que está en la fracción deseada
    df = df.loc[df['SessionId'] >= first_session] # Se guardan únicamente las sesiones cuyo Id sea mayor al Id encontrado
    
    # Nota: Este procedimiento sólo funciona correctamente si el dataset está ordenado según SessionId
    
    itemids = df['ItemId'].unique()
    n_items = len(itemids)

    print('Fractioned data set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(df), df.SessionId.nunique(), df.ItemId.nunique()))
    filename = path + '{}_{}.tsv'.format(prefix, fraction)
    df.to_csv(filename, sep='\t', index=False)
    print("Saved as {}".format(filename))

In [ ]:
# Para el propósito de esta demo, utilizamos una fracción del dataset de entrenamiento.
fraction = 64
train = pd.read_csv(dst_path + 'train.tsv', sep='\t', dtype={'ItemId':np.int64})
train_fraction(train, fraction)
train_frac = pd.read_csv(dst_path + 'train_64.tsv', sep='\t', dtype={'ItemId':np.int64})

In [ ]:
!python src/run.py ./data/train_64.tsv -t ./data/test.tsv -m 1 5 10 20 -ps layers=100,batch_size=32,dropout_p_embed=0.1,dropout_p_hidden=0,learning_rate=0.1,momentum=0,n_sample=2048,sample_alpha=0.75,bpreg=0,logq=1,loss=cross-entropy,constrained_embedding=True,elu_param=0,n_epochs=200 -d cuda:0

### Actividad 4: Recomendación en base a sesiones.

Reporte los resultados del entrenamiento, los valores de Recall y MRR obtenidos.
Explique, en sus palabras, el desafío de recomendación en base a sesiones. Cómo difiere de los desafíos que hemos visto hasta ahora, y cómo se evalúa.

#### Respuesta:

Ingrese su respuesta continuación. Recuerde terminar su respuesta con una celda de texto con sus conclusiones y respuestas.

### Actividad 5: Análisis de relevancia de secuencia.

Gru4Rec, al ser un recomendador secuencial, utiliza la información de la secuencia de interacciones del usuario para predecir el siguiente elemento. En este ejercicio veremos si el orden en que el usuario interactuó con los ítems realmente influye en el rendimiento de Gru4Rec.

Para testear esto, vamos a modificar la base de datos, cambiando el orden en que los usuarios interactuaron con los items en el set de entrenamiento. Para lograr esto, usted simplemente debe:

* Modificar la columna 'Time' del dataframe 'train_frac' con tal de que el orden en que los usuarios interactuaron con los items sea distinto al original. Esto lo puede lograr de distintas formas; asignándole un valor aleatorio a la columna Time, invirtiéndo los valores de Time en el dataframe, etc.
* Importante: Solo modifique el orden en que los usuarios interactuaron con los ítems. No debe agregar, eliminar ni intercambiar los ítems que fueron vistos por cada usuario. Cada usuario debe haber interactuado con exactamente los mismos items con los que interactuó originalmente, pero en un orden diferente.
* Guarde el dataframe en un archivo .tsv.
* Entrene el modelo Gru4Rec sobre su nuevo dataset. Todos los hiperparámetros deben ser los mismos que en el entrenamiento original, para que este sea el único cambio.

Explique su estrategia de modificación del dataset y reporte los resultados, tanto de recall como de MRR y compárelos con los resultados del dataset original. Convluya a partir de esto la importancia del orden de las interacciones en Gru4Rec.

#### Respuesta:

Ingrese su respuesta continuación. Recuerde terminar su respuesta con una celda de texto con sus conclusiones y respuestas.